# IMDB Sentiment Analysis

In [50]:
import tarfile
with tarfile.open('../../src/resources/imdb/aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()
    tar.close()

In [51]:
import os
import sys
import pyprind
import pandas as pd

# change the 'basepath' to the directory of the
# unzipped movie dataset

basepath = './aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()

for s in ('test', 'train'):
  for i in ('pos', 'neg'):
    path = os.path.join(basepath, s, i)
    for file in sorted(os.listdir(path)):
      with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
        txt = infile.read()

      new_row = pd.DataFrame([[txt, labels[i]]], columns=['review', 'sentiment'])
      df = pd.concat([df, new_row], ignore_index=True)
      pbar.update()

df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


In [52]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('../../src/resources/imdb/movie_data.csv', index=False, encoding='utf-8')

In [53]:
df = pd.read_csv('../../src/resources/imdb/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [54]:
# Does dataframe have 50,000 rows & 2 columns?
df.shape

(50000, 2)

## Bag of Words

1. Created a vocabulary of unique tokens that represent the each word in the dataset. In some instances it may be phrases, but here we will use words.
2. Construct feature vectors from each document that contains the counts of how often each word occurs in the particular document.
3. Since each document only contains a small subset of the words in the vocabulary, the feature vectors are considered "sparse".

### Things to consider with sparse feature vectors to avoid the curse of dimensionality & overfitting:
1. Feature selection
2. Feature extraction
3. Regularization
4. Dimensionality reduction


In [55]:
# Transform the text into a bag of words
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining, the weather is sweet, and one and one is two'])

bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())


{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### Term Frequency-Inverse Document Frequency (TF-IDF)

Often times in documents, we'll find words that appear often across many documents. These words are not particularly useful for distinguishing between documents.

TF-IDF is a way to weight the importance of each word in a document. It is a combination of the term frequency (TF) and the inverse document frequency (IDF).

TF is the number of times a word appears in a document divided by the total number of words in the document.

IDF is the logarithm of the total number of documents divided by the number of documents that contain the word.

TF-IDF is the product of TF and IDF.
 
$tf\text{-}idf(t,d) = tf(t,d) \cdot idf(t,d)$

$idf(t,d) = \log(\frac{n_d}{1+df(d,t)})$

where:
- $tf(t,d)$ = frequency of term $t$ in document $d$
- $df(d,t)$ = number of documents containing term $t$
- $n_d$ = total number of documents
- Adding the constant of 1 to the denominator of the IDF formula prevents division by zero.
- log is to ensure that the low document frequencies are not given too much weight.

In [56]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                          norm='l2',
                          smooth_idf=True)

np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


As we can see the word "is" appears in all three documents, so it has a relatively low TF-IDF score.

If you were to manually calculate the TF-IDF scores, you would see that the sklearn implementation is slightly different.

$idf(t,d) = \log(\frac{1+n_d}{1+df(d,t)})$

$tf-idf(t,d) = tf(t,d) \cdot (idf(t,d) + 1)$

The +1 in the TF-IDF formula is due to the smooth_idf parameter.

### Normalization

While it is normal to normalize the feature vectors, it is not necessary for the sklearn TF-IDF transformation as it already normalizes with the parameter `norm='l2'`.

$v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_1^2 + v_2^2 + \cdots + v_n^2}} = \frac{v}{\sqrt{\sum_{i=1}^n v_i^2}}$

### TF-IDF of "is" example:

"is" has a term frequency of 3 in the third document.

$tf = 3$

"is" appears in 3 documents, so the document frequency is 3.

$df = 3$

$idf("is", d_3) = \log(\frac{1+3}{1+3}) = \log(1) = 0$

$tf-idf("is", d_3) = tf \cdot (idf + 1) = 3 \cdot (0 + 1) = 3$

Computing TF-IDF for all terms in the third document returns:

`[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0, 1.69, 1.29]`

After normalization, the TF-IDF scores are:

$tf-idf(d_3)_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0, 1.69, 1.29]}{\sqrt{3.39^2 + 3.0^2 + 3.39^2 + 1.29^2 + 1.29^2 + 1.29^2 + 2.0^2 + 1.69^2 + 1.29^2}}$
$= [0.5,0.45,0.5,0.19,0.19,0.19,0.3,0.25,0.19]$

$tf-idf("is", d_3) = 0.45$

### Cleaning the text

Before we utilize the bag of words on the real dataset, we want to remove all unwanted characters.

In [57]:
# See how the characters printed out have HTML tags
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In this project, we will remove HTML tags, punctuations (although punctuations are usually useful for sentiment analysis), and none words characters, capitalization (even though it is useful in context where it is semantically meaningful [not at the beginning of a sentence]).

We will keep emojis (such as `:)`). We will remove the nose `-` character from emojis.

In [58]:
import re

def preprocessor(text):
  text = re.sub(r'<[^>]*>', '', text)
  emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                        text)
  text = (re.sub(r'[\W]+', ' ', text.lower()) +
          ' '.join(emoticons).replace('-', ''))
  return text

print(preprocessor(df.loc[0, 'review'][-50:]))

print(preprocessor("</a>This :) is :( a test :-)!"))


is seven title brazil not available
this is a test :) :( :)


In [59]:
# Apply the preprocessor to the dataset
df['review'] = df['review'].apply(preprocessor)
df.head(3)

,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0


In [60]:
# Split the text corpora into individual elements
# Splitting on whitespace of the cleaned text is one way
def tokenizer(text):
  return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

Another useful NLP tool, is word stemming.

Stemming is the process of reducing words to their root form and mapping words to their same stem.

### Note:

Porter stemming is one of the oldest and simplest stemming algorithms. There are other algorithms such as `Snowball` and `Lancaster`.

While both Snowball and Lancaster are both faster than the original Porter, the Lancaster stemmer is also notorious for being more aggressive than the Porter stemmer.

### Lemmatization

While stemming can create non-real words, such as "thu", lemmatization can obtain the canonical froms of the individual words.

Lemmatization is more sophisticated and computationally complex than stemming and has been observed to have little impact on performance of text classification.


In [61]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

### Stop Word Filtering

Stop words are common words that bear little or no useful information that can be used to distinguish between documents.

Examples of stop words are: "the", "is", "at", "it", etc.

Removing stop words can be useful if we are working with raw or normalized term frequencies rather than TF-IDF, which are already downweighting frequently occurring words.

Generally speaking, TF-IDF is preferred over raw term frequencies. But here are a few reasons why we might go with raw or normalized term frequencies:

1. We want to reduce computational complexity (usually not a problem now in days).
2. IDF component might actually remove important signal in short texts.

In [62]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

# Load english stop words
stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brandonfeist/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

## Training a logistic regression model for document classification

In [63]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [66]:
# Using GridSearchCV to find the best parameters for the logistic regression model
# WARNING: Running this GridSearch will take a long time to complete.
# We limited the number of parameter combinations, since the number of feature vectors, as well as the larger vocabulary size, can make the GridSearch computationally expensive.
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        token_pattern=None)

param_grid = [{'vect__ngram_range': [(1,1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer,
                                    tokenizer_porter],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]},
                {'vect__ngram_range': [(1,1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer,
                                    tokenizer_porter],
                'vect__use_idf':[False],
                'vect__norm':[None],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]}
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf',
                     LogisticRegression(random_state=0,
                                        solver='liblinear'))])

# Using n_jobs=-1 to use all available CPU cores (for parallel processing)
# If bugging, set n_jobs=1
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                            scoring='accuracy',
                            cv=5, verbose=1,
                            n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/Users/brandonfeist/git/ml/python-machine-learning-book/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(
/Users/brandonfeist/git/ml/python-machine-learning-book/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(
/Users/brandonfeist/git/ml/python-machine-learning-book/venv/lib

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x328f0b560>} 


As you can see by the GridSearch output, we have the best parameters using the regular tokenizer without Porter stemming, no stop-word library, and TF-IDF in combination with logistic regression classifier that uses L2-regularization, with a regularization strength of 10.

In [67]:
# Avg 5-fold cross-validation accuracy scores on the training dataset and classification accuracy on the test dataset
# The results show the model can predict if a movie review is positive or negative with 90% accuracy.
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

clf = gs_lr_tfidf.best_estimator_

print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

CV Accuracy: 0.897
Test Accuracy: 0.899
